In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importing necessary modules
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from datetime import datetime

In [ ]:
SPLIT=0.8
PATH = "/content/drive/MyDrive/Cognizant/Task 2/"
K = 10

In [ ]:
# Function to load data from path
def load_data(tablename):
    df = pd.read_csv(PATH+tablename)
    df.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')
    return df


In [ ]:
# Function to split dependent and independent variables
def split_columns(data: pd.DataFrame):
    # Check to see if the target variable is present in the data
    if "estimated_stock_pct" not in data.columns:
        raise Exception(f"Target: estimated_stock_pct is not present in the data")

    X = data.drop(columns=["estimated_stock_pct"])
    y = data["estimated_stock_pct"]
    return X, y



In [ ]:
# Train using cross validation
def train(X: pd.DataFrame, y: pd.Series):

    # Create a list that will store the accuracies of each fold
    accuracy = []

    # Enter a loop to run K folds of cross-validation
    for fold in range(0, K):

        # Instantiate algorithm and scaler
        model = RandomForestRegressor()
        scaler = StandardScaler()

        # Create training and test samples
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=SPLIT, random_state=42)

        # Scale X data, we scale the data because it helps the algorithm to converge
        # and helps the algorithm to not be greedy with large values
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Train model
        trained_model = model.fit(X_train, y_train)

        # Generate predictions on test sample
        y_pred = trained_model.predict(X_test)

        # Compute accuracy, using mean absolute error
        mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)
        accuracy.append(mae)
        print(f"Fold {fold + 1}: MAE = {mae:.3f}")

    # Finish by computing the average MAE across all folds
    print(f"Average MAE: {(sum(accuracy) / len(accuracy)):.2f}")

In [ ]:
# Converting TimeStamp to Date Time Format and Hourly
def preprocess_timestamp(df: pd.DataFrame):
  df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
  dummy = df.copy()
  new_ts = dummy["timestamp"].tolist()
  new_ts = [i.strftime('%Y-%m-%d %H:00:00') for i in new_ts]
  new_ts = [datetime.strptime(i, '%Y-%m-%d %H:00:00') for i in new_ts]
  dummy["timestamp"] = new_ts
  return dummy

In [ ]:
# Data Aggregation based on timestamp and product id
def aggregate(df: pd.DataFrame, tablename: str):

  if tablename =="sales":
    sales_agg = df.groupby(['timestamp', 'product_id']).agg({'quantity': 'sum'}).reset_index()
    return sales_agg

  elif tablename == "stock":
    stock_agg = df.groupby(['timestamp', 'product_id']).agg({'estimated_stock_pct': 'mean'}).reset_index()
    return stock_agg

  elif tablename == "temp":
    temp_agg = df.groupby(['timestamp']).agg({'temperature': 'mean'}).reset_index()
    return temp_agg
  else:
    print("Provide accurate table name. Empty DataFrame Returned.")
    return pd.DataFrame()

In [ ]:
# Merging DataFrames. Categories not included as they have least effect on the model
# Only Timestamp Hour is used as it largely effects. Timestamp Day of week is also used. Day of Month is not used as
# the importance of day of week and month is same
def merge_dfs(sales_agg: pd.DataFrame, stock_agg: pd.DataFrame, temp_agg: pd.DataFrame,sales_df: pd.DataFrame):
  merged_df = stock_agg.merge(sales_agg, on=['timestamp', 'product_id'], how='left')
  merged_df = merged_df.merge(temp_agg, on='timestamp', how='left')
  merged_df['quantity'] = merged_df['quantity'].fillna(0)

  product_price = sales_df[['product_id', 'unit_price']]
  product_price = product_price.drop_duplicates()
  merged_df = merged_df.merge(product_price, on="product_id", how="left")
  merged_df['timestamp_day_of_week'] = merged_df['timestamp'].dt.dayofweek
  merged_df['timestamp_hour'] = merged_df['timestamp'].dt.hour
  merged_df.drop(columns=['timestamp'], inplace=True)
  merged_df.drop(columns=['product_id'], inplace=True)
  return merged_df

In [ ]:
def main():
  # Data Loading
  sales_df = load_data("sales.csv")
  stock_df = load_data("sensor_stock_levels.csv")
  temp_df = load_data("sensor_storage_temperature.csv")
  # Preprocessing
  sales_df = preprocess_timestamp(sales_df)
  stock_df = preprocess_timestamp(stock_df)
  temp_df = preprocess_timestamp(temp_df)
  # Aggregation
  sales_agg = aggregate(sales_df, "sales")
  stock_agg = aggregate(stock_df, "stock")
  temp_agg = aggregate(temp_df, "temp")
  # Merged Data Frame
  merged_df = merge_dfs(sales_agg, stock_agg, temp_agg,sales_df)
  # Splitting Dependent and Independent Columns
  x,y = split_columns(merged_df)
  # Training and Evaluation
  train(x, y)


In [ ]:
if __name__ == "__main__":
    main()

Fold 1: MAE = 0.244
Fold 2: MAE = 0.245
Fold 3: MAE = 0.244
Fold 4: MAE = 0.245
Fold 5: MAE = 0.244
Fold 6: MAE = 0.244
Fold 7: MAE = 0.245
Fold 8: MAE = 0.244
Fold 9: MAE = 0.245
Fold 10: MAE = 0.244
Average MAE: 0.24
